# MNIST 손글씨 - CNN1

- 2 Conv2d layers  
- 1 Fully connected Network layer  
- 1 Output layer

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
 
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 전처리

In [2]:
from tensorflow.keras.datasets import mnist
(X_train, y_train),(X_test,y_test) = mnist.load_data()

0에서 1사이의 값으로 정규화 

In [3]:
X_train = X_train / 255.0
X_test = X_test / 255.0

3차원 데이터를 4차원으로 - 2차원은 3차원으로 

In [4]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

X_train.shape, X_test.shape


((60000, 28, 28, 1), (10000, 28, 28, 1))

원 핫 인코딩 

In [5]:
from tensorflow.keras.utils import to_categorical

Y_train = to_categorical(y_train,10)
Y_test = to_categorical(y_test,10)

Y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

### 모델 정의, 설정, 학습, 평가

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [7]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), input_shape = (28,28,1), activation='relu'),
    Conv2D(64, (3,3),activation='relu'),
    Flatten(),                                                              # 3차원을 1차원으로 변환
    Dense(512,activation='relu'),
    Dense(10,'softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               18874880  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 18,898,826
Trainable params: 18,898,826
Non-trainable params: 0
_________________________________________________________________


In [8]:
# 모델 실행 환경
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

모델 최적화를 위한 설정


In [9]:
model_path = 'model/best_mnist_cnn_1.h5'
checkpoint = ModelCheckpoint(
    model_path, verbose=0, save_best_only=True
)
early_stopping = EarlyStopping(patience=10)

In [15]:
history = model.fit(
    X_train, Y_train, validation_split=0.2,
    epochs=30, batch_size=200, verbose=0,
    callbacks=[checkpoint, early_stopping]
)

In [17]:
from tensorflow.keras.models import load_model
best_model=load_model(model_path)
best_model.evaluate(X_test, Y_test)

[0.06827908009290695, 0.9796000123023987]